In [2]:
import json
import numpy as np
import pandas as pd
from collections import defaultdict
from typing import List
import re
# %matplotlib widget

In [11]:
#######################################################
# flatten
####################################################### 

def flatten2list(object):
    """ This function flattens objects in a nested structure """
    gather = []
    for item in object:
        if isinstance(item, (list, set)):
            gather.extend(flatten2list(item))            
        else:
            gather.append(item)
    return gather

def flatten2set(object) -> set:
    """ This function flattens objects in a nested structure and return a set"""

    return set(flatten2list(object))

#######################################################
# epitope dataset
####################################################### 
def dispatch_dict(operator, x, value):
    return {
    '==': lambda: x == value,
    ">": lambda:  x > value,
    "<": lambda:  x < value,
    }.get(operator, lambda: None)()

#######################################################
# epitope dataset
####################################################### 
class Epitope_DB:
    
    def __init__(self, path:str='../data/20201123_EpitopevsHLA.pickle'):
        self.df = pd.read_pickle(path)
        
    def __repr__(self):
        return f""" Epitope_DB(records={len(self.df)}, columns={self.df.columns}) """
    
    def __str__(self):
        return __repr__()
    
    def epitope(self, value):
        if isinstance(value, str):
            ind = self.df.Epitope == value 
        else: 
            ind = self.df.Epitope.apply(lambda x: x in value) 
        self.df = self.df[ind]
        return self
    
    def ellipro(self, value):
        if isinstance(value, str):
            ind = self.df.Epitope == value 
        else: 
            ind = self.df['ElliPro Score'].apply(lambda x: x in value) 
        self.df = self.df[ind]
        return self
    
    def hlavsep(self, hla_allel:str='Luminex Alleles'):
        import pandas as pd
        from collections import defaultdict
        
        hlas = flatten2set(self.df[hla_allel].values)
        hlavsep_dict = defaultdict(list)
        for hla in hlas:
            ind = self.df[hla_allel].apply(lambda x: hla in x)
            epitopes = flatten2set(self.df[ind]['Epitope'].values)
            hlavsep_dict['HLA'].append(hla)
            hlavsep_dict['Epitope'].append(epitopes)
        self._hlavsep = pd.DataFrame(hlavsep_dict)
        return self._hlavsep

    def min_hlavsep(self, epitopes):
        _epitopes = epitopes.copy()
        hlavsep_df = self.hlavsep()
        hla_ep = defaultdict(set)
        while len(_epitopes) != 0:
            ind_max = hlavsep_df.Epitope.apply(lambda x: len(x.intersection(_epitopes))).sort_values().index[-1]
            hla = hlavsep_df.iloc[ind_max].HLA
            ep = hlavsep_df.iloc[ind_max].Epitope.intersection(_epitopes)
            hla_ep[hla] = ep
            _epitopes.difference_update(ep)
        return dict(hla_ep)
    
#######################################################
# desa dataset
####################################################### 
class DESA_DB:
    
    def __init__(self, path:str='../data/desa_3d_view.pickle'):
        self.df = pd.read_pickle(path)
        
    def __repr__(self):
        return f""" DESA_DB(records={len(self.df)}, columns={self.df.columns}) """
    
    def __str__(self):
        return __repr__()
    
    def donor_type(self, donor_type:str='Deceased'):
        if donor_type not in ['Living', 'Deceased']:
            raise KeyError(f'{donor_type} does not exist in the df values, accepted values: {self.df.Donor_Type.unique()}')
        ind = self.df.Donor_Type.apply(lambda x: x == donor_type)                          
        self.df = self.df[ind]
        return self
        
    def hla_class(self, hla_class):
        if hla_class not in ['I', 'II', 'I,II']:
            raise KeyError(f'{hla_class} does not exist in the df values, accepted values: {self.df.Donor_HLA_Class.unique()}')
        ind = self.df.Donor_HLA_Class.apply(lambda x: x == hla_class)                          
        self.df = self.df[ind]
        return self
        
    def early_failed(self, threshold):
        ind_t = self.df['Survival[Y]'].apply(lambda x: x < threshold)
        ind_e = self.df.Failure.apply(lambda x: x == 1)
        self.df = self.df[ind_t & ind_e]
        return self

    def late_failed(self, threshold):
        ind_t = self.df['Survival[Y]'].apply(lambda x: x > threshold)
        ind_e = self.df.Failure.apply(lambda x: x != 1 )
        self.df = self.df[ind_t & ind_e]
        return self
    
  
help(str)

Help on class str in module builtins:

class str(object)
 |  str(object='') -> str
 |  str(bytes_or_buffer[, encoding[, errors]]) -> str
 |  
 |  Create a new string object from the given object. If encoding or
 |  errors is specified, then the object must expose a data buffer
 |  that will be decoded using the given encoding and error handler.
 |  Otherwise, returns the result of object.__str__() (if defined)
 |  or repr(object).
 |  encoding defaults to sys.getdefaultencoding().
 |  errors defaults to 'strict'.
 |  
 |  Methods defined here:
 |  
 |  __add__(self, value, /)
 |      Return self+value.
 |  
 |  __contains__(self, key, /)
 |      Return key in self.
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 |  __format__(self, format_spec, /)
 |      Return a formatted version of the string as described by format_spec.
 |  
 |  __ge__(self, value, /)
 |      Return self>=value.
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  

In [4]:
desa_db = DESA_DB()
desa_early = desa_db.hla_class('I').donor_type('Deceased').early_failed(1/6).df
ep_early = set(flatten2list([set(item.keys()) for item in flatten2list(desa_early.EpvsHLA_Donor.values)]))

desa_db = DESA_DB()
desa_late = desa_db.hla_class('I').donor_type('Deceased').late_failed(10).df
ep_late = set(flatten2list([set(item.keys()) for item in flatten2list(desa_late.EpvsHLA_Donor.values)]))

relevant_ep = ep_early - ep_late

# ep_early
# len(desa_early)
# desa_early.
# desa_db.df
# desa_late
# ep_late
relevant_ep, len(relevant_ep)


({'105S',
  '113HN',
  '114H',
  '114Q',
  '116L',
  '131S',
  '144QL',
  '44RME',
  '62EE',
  '62QE',
  '63NI',
  '65QIA',
  '66IS',
  '66IY',
  '66NH',
  '70IAQ',
  '71TD',
  '74Y',
  '77D',
  '99S',
  '9H'},
 21)

In [6]:
desa_db.df

,TransplantID,Status,DESA_Epitope,#DESA,EpvsHLA_Pos,EpvsHLA_Donor,Failure,Survival[Y],DESA->Donor_HLA,Donor_HLA,Donor_HLA_Class,Donor_Type
17,2186,DESA,"{62GE, 144TKH, 65RK, 145KHA, 95V, 62GK, 107W}",7,"{'95V': {'A*69:01', 'A*02:03', 'A*02:01'}, '14...","{'65RK': 'A*02:01', '62GE': 'A*02:01', '145KHA...",2,13.010959,{'A*02:01': 7},{A*02},I,Deceased
36,5226,DESA,"{73TVS, 173K}",2,"{'173K': {'C*03:03', 'C*03:04'}, '73TVS': {'C*...","{'73TVS': 'C*03:04', '173K': 'C*03:04'}",0,12.008219,{'C*03:04': 2},{C*03},I,Deceased
38,248,DESA,"{41T, 45KE}",2,"{'45KE': {'B*44:03', 'B*40:01', 'B*40:02', 'B*...","{'45KE': 'B*40:01', '41T': 'B*40:01'}",0,15.010959,{'B*40:01': 2},{B*40},I,Deceased
50,3800,DESA,"{199V, 163LG, 166ES, 162GLS}",4,"{'162GLS': {'B*45:01', 'B*44:02', 'B*44:03'}, ...","{'163LG': 'B*44:02', '199V': 'B*44:02', '166ES...",0,16.010959,{'B*44:02': 4},{B*44},I,Deceased
71,1545,DESA,"{144TKH, 65RK, 145KHA, 95V, 62GK, 107W}",6,"{'95V': {'A*69:01', 'A*02:03', 'A*02:01'}, '14...","{'65RK': 'A*02:01', '145KHA': 'A*02:01', '62GK...",2,15.106849,{'A*02:01': 6},{A*02},I,Deceased
82,1071,DESA,"{71ATD, 76ED}",2,"{'71ATD': {'B*27:03', 'B*27:05'}, '76ED': {'B*...","{'71ATD': 'B*27:05', '76ED': 'B*27:05'}",2,14.556164,{'B*27:05': 2},{B*27},I,Deceased
83,1072,DESA,"{97S, 180E, 177DT, 76ESN, 71TTS}",5,"{'177DT': {'B*41:01', 'B*08:01', 'B*42:01'}, '...","{'71TTS': 'B*08:01', '76ESN': 'B*08:01', '180E...",0,18.010959,{'B*08:01': 5},{B*08},I,Deceased
100,1218,DESA,{62LQ},1,"{'62LQ': {'A*43:01', 'A*29:02'}}",{'62LQ': 'A*29:02'},0,19.010959,{'A*29:02': 1},{A*29},I,Deceased
107,1402,DESA,"{156QA, 76VRN, 91R, 151AHA, 76VS, 73TVS, 173K,...",8,"{'151AHA': {'A*11:01', 'A*11:02'}, '156QA': {'...","{'151AHA': 'A*11:01', '163RW': 'A*11:01', '156...",0,16.021918,"{'A*11:01': 3, 'C*03:03': 5}","{C*03, A*11}",I,Deceased
108,1425,DESA,"{156QA, 163R, 152HA, 151AHA, 152A, 163RW}",6,"{'151AHA': {'A*11:01', 'A*11:02'}, '156QA': {'...","{'151AHA': 'A*11:01', '152A': 'A*11:01', '152H...",0,12.008219,{'A*11:01': 6},{A*11},I,Deceased


In [50]:
class Epitope_DB:
    
    def __init__(self, path:str='../data/20201123_EpitopevsHLA.pickle'):
        path = '/Users/Danial/Repos/dash_hla_3d/data/20201123_EpitopevsHLA.pickle' \
        
        self.ep_df = pd.read_pickle(path)

class Vis(Epitope_DB):
    
    def __init__(self):
        super().__init__()
    
    def do(self):
        return Vis().ep_df
        
# Epitope_DB()
class Danial:
    def __init__(self):
        pass
    
    @staticmethod
    def add(a, b):
        return a + b

    def _print(self, a, b):
        return self.add(a,b)
    
Danial()._print(2, 4)
whos

NameError: name 'whos' is not defined

In [15]:
desa_db = DESA_DB()
desa_early = desa_db.hla_class('I,II').donor_type('Deceased').early_failed(1/6).df
desa_early

,TransplantID,Status,DESA_Epitope,#DESA,EpvsHLA_Pos,EpvsHLA_Donor,Failure,Survival[Y],DESA->Donor_HLA,Donor_HLA,Donor_HLA_Class,Donor_Type
70,3876,DESA,"{45GV, 161D, 55PPA, 185I}",4,"{'55PPA': {'DQB1*03:02'}, '161D': {'A*03:01'},...","{'185I': 'DQB1*03:02', '55PPA': 'DQB1*03:02', ...",1,0.000000,"{'DQB1*03:02': 3, 'A*03:01': 1}","{A*03, DQB1*03}","I,II",Deceased
146,1903,DESA,"{55R, 55RPD, 67F, 70DA, 56PD, 70DRA, 96EV, 46V...",35,"{'275EL': {'A*30:01', 'A*03:01', 'A*11:01', 'A...","{'275EL': 'A*03:01', '161D': 'A*03:01', '97I':...",1,0.013699,"{'A*03:01': 4, 'DQB1*06:02': 17, 'DRB5*01:01':...","{DRB1*15, A*03, DQB1*06, DRB5*01}","I,II",Deceased
246,363,DESA,"{48Q, 96QN, 52LL, 30G[B]}",4,"{'48Q': {'DRB4*01:01'}, '52LL': {'DQB1*02:01',...","{'52LL': 'DQB1*02:01', '30G[B]': 'B*18:01', '9...",1,0.016438,"{'DQB1*02:01': 1, 'B*18:01': 1, 'DRB4*01:01': 2}","{DQB1*02, B*18, DRB4*01}","I,II",Deceased
427,904,DESA,"{16Y, 166ES, 94I, 95I, 163LE, 199V, 163LG, 96H...",13,"{'149H': {'DRB1*03:01', 'DRB1*14:04', 'DRB1*12...","{'16Y': 'DRB1*08:01', '149H': 'DRB1*08:01', '9...",1,0.000000,"{'DRB1*08:01': 5, 'B*44:02': 8}","{DRB1*08, B*44}","I,II",Deceased


In [16]:
ep_db = Epitope_DB()
hlavsep = ep_db.min_hlavsep(relevant_ep)
hlavsep

{'B*55:01': {'113HN', '116L', '131S', '63NI', '65QIA', '66IY', '70IAQ'},
 'A*31:01': {'105S', '114Q', '44RME', '62QE', '66NH', '77D'},
 'B*37:01': {'66IS', '71TD', '74Y', '99S', '9H'},
 'A*23:02': {'114H', '62EE'},
 'B*13:02': {'144QL'}}

In [17]:
ep_db = Epitope_DB()
hlavsep = ep_db.hlavsep()
_relevant_ep = relevant_ep.copy()
df_sorted = hlavsep.Epitope.apply(lambda x: len(x.intersection(_relevant_ep))).sort_values(ascending=False)
ind = df_sorted.apply(lambda x: x == 6) 
hlavsep[ind].Epitope.apply(lambda x: x.intersection(_relevant_ep))
# hlavsep.iloc[ind].Epitope.apply(lambda x: x.intersection(_relevant_ep))

<ipython-input-17-7b8cb946839d>:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  hlavsep[ind].Epitope.apply(lambda x: x.intersection(_relevant_ep))


5        {131S, 66IS, 144QL, 74Y, 113HN, 116L}
31        {44RME, 66NH, 105S, 77D, 62QE, 114Q}
55          {131S, 77D, 66IS, 74Y, 114H, 71TD}
97       {131S, 66IS, 144QL, 74Y, 113HN, 116L}
165    {131S, 63NI, 70IAQ, 66IY, 113HN, 65QIA}
Name: Epitope, dtype: object

# Cartesian Visualisation:

In [18]:
def load_epitope_db():
    """This function loads the Epitopes from different tables
    """
    desa = pd.read_pickle('../data/20201123_EpitopevsHLA.pickle')
    return desa

def load_desa_db():
    """This function loads the Epitopes from different tables
    """
    desa = pd.read_pickle('../data/desa_db.pickle')
    return desa

#######################################################
# get HLA locus
#######################################################
def get_hla_locus(hla:str) -> str:
    """ get the long locus (max 3 letters of gene) of hla """

    gene = hla.split('*')[0]
    return gene if len(gene) == 1 else gene[0:3]

#######################################################
# get HLA class
#######################################################
def get_hla_class(locus:str) -> str:
    """ get the hla class of a hla locus """

    hla_class = {'A':'Class_I', 'B':'Class_I', 'C':'Class_I', 
                 'DRB':'Class_II', 'DQA':'Class_II', 'DQB':'Class_II'}
    return hla_class.get(locus, 'Locus not found')

    
#######################################################
# get HLA class from path
#######################################################
def get_hla_class_from_path(path:str) -> str:
    """ get the long locus (max 3 letters of gene) of hla """
    
    hla_class = {'A':'Class_I', 'B':'Class_I', 'C':'Class_I', 
                 'DRB':'Class_II', 'DQA':'Class_II', 'DQB':'Class_II'}
    hla = path.split('/')[-1].split('_')[0]
    locus = get_hla_locus(hla)
    return hla_class[locus]

#######################################################
# pars pdb file
#######################################################
def pdb_parser(pdb_path:list):
    # Read PDB file to create atom/bond information
    with open(pdb_path, 'r') as infile:
        # store only non-empty lines
        lines = [l.strip() for l in infile if l.strip()]
    
    hla_class = get_hla_class_from_path(pdb_path)
    data = defaultdict(lambda: defaultdict(list))
    # Variables that store the character positions of different
    # parameters from the molecule PDB file
    serialpos = slice(6,11) 
    atm_namepos = slice(12, 16) 
    r_namepos = slice(17, 20) 
    chainpos = slice(21, 22) 
    r_idpos = slice(22, 26) 
    xpos = slice(30, 38) 
    ypos = slice(38, 46) 
    zpos = slice(46, 54) 
    occupos = slice(54, 60) 
    bfacpos = slice(60, 66) 
    atm_typepos = slice(77,79)

    for l in lines:
        line = l.split()
        if line[0] in ["ATOM", "HETATM"]:
            chain = l[chainpos].strip()
            x = float(l[xpos])
            y = float(l[ypos])
            z = float(l[zpos])
            data[hla_class][chain].append(np.array([x, y, z]))
    return data
        

#######################################################
# 3D to 2D Projection 
#######################################################
def loc_2_coord(atoms:List[np.array], dis:int=60):
    x, y, z = [], [], []
    for atom in atoms:
        x.append(atom[0])
        y.append(atom[1])
        z.append(atom[2])
        
    proj_xy = (np.array(x), np.array(y), -dis * np.ones(len(atoms)))
    proj_xz = (np.array(x),  -dis * np.ones(len(atoms)), np.array(z))
    proj_yz = (-dis * np.ones(len(atoms)), np.array(y), np.array(z))
    return proj_xy, proj_xz, proj_yz


#######################################################
# Visualise Projection 
#######################################################

def visualise_proj(data, hla_class, ep_coord):
    
    chain_colors = {
            "A": "#65A5E2", # Blue
            "B": "#CA7FE5", # purple
            "C": "#65E2AB", # green
    }
    
    import matplotlib as mpl
    from mpl_toolkits.mplot3d import Axes3D
    import numpy as np
    import matplotlib.pyplot as plt
    
    mpl.rcParams['legend.fontsize'] = 10

    fig = plt.figure()
    ax = fig.gca(projection='3d')
    alpha = 0.02
    
    proj_xy, proj_xz, proj_yz = loc_2_coord(data[hla_class]['A'])
    ax.scatter(proj_xy[0], proj_xy[1], proj_xy[2], s=10, c=chain_colors['A'], alpha=alpha)
    ax.scatter(proj_xz[0], proj_xz[1], proj_xz[2], s=10, c=chain_colors['A'], alpha=alpha)
    ax.scatter(proj_yz[0], proj_yz[1], proj_yz[2], s=10, c=chain_colors['A'], alpha=alpha)

    proj_xy, proj_xz, proj_yz = loc_2_coord(data[hla_class]['B'])
    ax.scatter(proj_xy[0], proj_xy[1], proj_xy[2], s=10, c=chain_colors['B'], alpha=alpha)
    ax.scatter(proj_xz[0], proj_xz[1], proj_xz[2], s=10, c=chain_colors['B'], alpha=alpha)
    ax.scatter(proj_yz[0], proj_yz[1], proj_yz[2], s=10, c=chain_colors['B'], alpha=alpha)

    proj_xy, proj_xz, proj_yz = loc_2_coord(data[hla_class]['C'])
    ax.scatter(proj_xy[0], proj_xy[1], proj_xy[2], s=10, c=chain_colors['C'], alpha=alpha)
    ax.scatter(proj_xz[0], proj_xz[1], proj_xz[2], s=10, c=chain_colors['C'], alpha=alpha)
    ax.scatter(proj_yz[0], proj_yz[1], proj_yz[2], s=10, c=chain_colors['C'], alpha=alpha)
    
    proj_xy, proj_xz, proj_yz = loc_2_coord(ep_coord)
    ax.scatter(proj_xy[0], proj_xy[1], proj_xy[2], s=10, c='y')
    ax.scatter(proj_xz[0], proj_xz[1], proj_xz[2], s=10, c='y')
    ax.scatter(proj_yz[0], proj_yz[1], proj_yz[2], s=10, c='y')

    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    
    ax.set_title(f'HLA {hla_class}')
    plt.show()


        

In [19]:
pdb_path = '../data/HLAMolecule/A/A_01_01_V1.pdb'
# get_hla_class_from_path(pdb_path)
data = pdb_parser(pdb_path)

In [20]:
ep_db = load_epitope_db()
# desa_db = load_desa_db()
dict(ep_db[ep_db.Epitope == '71TTS'].Location.values[0])

{'B*15:10': [45.97, 87.81, 33.5],
 'B*14:02': [30.05, -6.31, 33.63],
 'B*48:01': [-4.68, -5.75, -33.05],
 'B*15:12': [29.95, 34.68, 33.67],
 'B*41:01': [4.6, 6.54, -33.68],
 'B*35:01': [21.05, 75.55, 75.86],
 'B*15:03': [29.86, 34.79, 33.55],
 'B*40:02': [-4.43, -6.38, -33.42],
 'B*78:01': [-4.73, 35.18, 22.02],
 'B*39:01': [-4.49, 5.55, 33.21],
 'B*50:01': [-4.44, -6.43, -33.42],
 'B*18:01': [4.45, 5.56, -33.02],
 'B*15:02': [29.91, 34.84, 33.4],
 'B*07:03': [4.73, 6.06, -33.27],
 'B*40:01': [4.74, 6.68, -33.81],
 'B*14:01': [30.21, -6.52, 33.85],
 'B*45:01': [4.45, 6.68, -33.59],
 'B*15:11': [46.0, 87.58, 32.99],
 'B*35:08': [30.11, 6.96, 21.63],
 'B*08:01': [-4.72, -6.16, -33.45],
 'B*15:18': [45.99, 87.65, 33.25],
 'B*40:06': [-4.52, -6.75, -33.62],
 'B*15:01': [29.95, 34.89, 33.3]}

In [21]:
desa_db = DESA_DB()
# # dir(desa_db.early_failed(1/4))
db = desa_db.early_failed(1/4)
# # desa_db.late_failed(10)
# db.df

In [22]:
def get_desa_coord(desa_df, ep_df):
    
    coord = []
    for ep_hla in desa_df[desa_df.Donor_HLA_Class == 'I'].EpvsHLA_Donor.values.tolist():
        for ep, hla in ep_hla.items():
            try:
                _coord = dict(ep_df[ep_df.Epitope == ep].Location.values[0])[hla]
                coord.append(_coord)
            except Exception as e:
                print(f'{type(e).__name__}: epitope {ep} does not have location for hla {hla}')
    return np.array(coord)

ep_coord = get_desa_coord(db.df, ep_db)                

KeyError: epitope 71TTS does not have location for hla B*39:06
KeyError: epitope 158T does not have location for hla B*39:06
KeyError: epitope 76ESN does not have location for hla B*39:06
KeyError: epitope 63NI does not have location for hla B*39:06


In [12]:
# plt.close('all')
# visualise_proj(data, 'Class_I')
visualise_proj(data, 'Class_I', ep_coord)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …